# Create and train MitoSplit-Net model

## Import required Python libraries



In [1]:
import util
import plotting
import training

import numpy as np
import matplotlib.pyplot as plt
plt.rc('xtick', labelsize=18)
plt.rc('ytick', labelsize=18)
plt.rc('axes', labelsize=20)
plt.rc('legend', fontsize=18)
from tqdm import tqdm

import tensorflow as tf

In [2]:
#Define GPU device where the code will run on
gpu = tf.config.list_physical_devices('GPU')[0]
print(gpu)
tf.config.experimental.set_memory_growth(gpu, True)
gpu = tf.device('GPU:0/')

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


## Data and models directories

In [3]:
base_dir = 'D:/Santi/'
print('base_dir:', base_dir)

data_path = base_dir+'Data/'
print('data_path:', data_path)

model_path = base_dir+'Models/'
print('model_path:', model_path)

base_dir: D:/Santi/
data_path: D:/Santi/Data/
model_path: D:/Santi/Models/


## Create model, split dataset and train

### No preprocessing, different batch sizes

#### Only mito channel as input

In [4]:
#Inputs
input_data = util.load_h5(data_path, 'Mito')
print('Inputs'+':', input_data.shape)

#Outputs
output_data = util.load_h5(data_path, 'Proc')
print('Outputs:', output_data.shape)


Loading //lebsrv2.epfl.ch/LEB_SHARED/SHARED/_Scientific projects/MitoSplit-Net/Data/Mito.h5
Converting to array
Inputs: (37000, 128, 128)

Loading //lebsrv2.epfl.ch/LEB_SHARED/SHARED/_Scientific projects/MitoSplit-Net/Data/Proc.h5
Converting to array
Outputs: (37000, 128, 128)


In [5]:
with gpu:
  nb_filters = 8
  firstConvSize = 9
  batch_size = [8, 16, 32, 256]
  model, history, frames_test = {}, {}, {}
  
  for b in batch_size:
    model_name = 'ref_f%i_c%i_b%i'%(nb_filters, firstConvSize, b)
    print('Model:', model_name)
    model[model_name] = training.create_model(nb_filters, firstConvSize)
    history[model_name], frames_test[model_name] = training.train_model(model[model_name], input_data, output_data, batch_size=b) 

Model: ref_f8_c9_b8
* Start Encoder Section *
* Start Center Section *
* Start Decoder Section *
Epoch 1/20
2960/2960 - 45s - loss: -1.9034e-01 - binary_accuracy: 0.9781 - val_loss: -3.6308e-01 - val_binary_accuracy: 0.9876
Epoch 2/20
2960/2960 - 45s - loss: -4.8994e-01 - binary_accuracy: 0.9820 - val_loss: -5.5039e-01 - val_binary_accuracy: 0.9827
Epoch 3/20
2960/2960 - 44s - loss: -5.5902e-01 - binary_accuracy: 0.9854 - val_loss: -5.6908e-01 - val_binary_accuracy: 0.9893
Epoch 4/20
2960/2960 - 44s - loss: -5.9835e-01 - binary_accuracy: 0.9873 - val_loss: -6.1088e-01 - val_binary_accuracy: 0.9925
Epoch 5/20
2960/2960 - 44s - loss: -6.2382e-01 - binary_accuracy: 0.9887 - val_loss: -6.1731e-01 - val_binary_accuracy: 0.9935
Epoch 6/20
2960/2960 - 44s - loss: -6.4301e-01 - binary_accuracy: 0.9896 - val_loss: -6.6040e-01 - val_binary_accuracy: 0.9902
Epoch 7/20
2960/2960 - 44s - loss: -6.5739e-01 - binary_accuracy: 0.9901 - val_loss: -6.7747e-01 - val_binary_accuracy: 0.9909
Epoch 8/20
296

In [6]:
folder_name = list(model.keys())

util.save_model(model, model_path, ['model']*len(model), folder_name)
util.save_pkl(history, model_path, ['history']*len(model), folder_name)
util.save_pkl(frames_test, model_path, ['frames_test']*len(model), folder_name)


Saving H:/Santi/Models/ref_f8_c9_b8/model.h5

Saving H:/Santi/Models/ref_f8_c9_b16/model.h5

Saving H:/Santi/Models/ref_f8_c9_b32/model.h5

Saving H:/Santi/Models/ref_f8_c9_b256/model.h5
Done.

Saving H:/Santi/Models/ref_f8_c9_b8/history

Saving H:/Santi/Models/ref_f8_c9_b16/history

Saving H:/Santi/Models/ref_f8_c9_b32/history

Saving H:/Santi/Models/ref_f8_c9_b256/history
Done.

Saving H:/Santi/Models/ref_f8_c9_b8/frames_test

Saving H:/Santi/Models/ref_f8_c9_b16/frames_test

Saving H:/Santi/Models/ref_f8_c9_b32/frames_test

Saving H:/Santi/Models/ref_f8_c9_b256/frames_test
Done.


#### Mito + Drp1 channels as inputs

In [7]:
#Inputs
input_data = util.load_h5(data_path, 'Mito')
input_data = np.stack((input_data, util.load_h5(data_path, 'Drp1')), axis=-1)
print('Inputs'+':', input_data.shape)

#Outputs
output_data = util.load_h5(data_path, 'Proc')
print('Outputs:', output_data.shape)


Loading D:/Santi/Data/Mito.h5
Converting to array

Loading D:/Santi/Data/Drp1.h5
Converting to array
Inputs: (37000, 128, 128, 2)

Loading D:/Santi/Data/Proc.h5
Converting to array
Outputs: (37000, 128, 128)


In [8]:
with gpu:
  nb_filters = 8
  firstConvSize = 9
  nb_input_channels = 2
  batch_size = [8, 16, 32, 256]
  model, history, frames_test = {}, {}, {}
  
  for b in batch_size:
    model_name = 'multich_ref_f%i_c%i_b%i'%(nb_filters, firstConvSize, b)
    print('Model:', model_name)
    model[model_name] = training.create_model(nb_filters, firstConvSize, nb_input_channels)
    history[model_name], frames_test[model_name] = training.train_model(model[model_name], input_data, output_data, batch_size=b) 

Model: multich_ref_f8_c9_b8
* Start Encoder Section *
* Start Center Section *
* Start Decoder Section *
Epoch 1/20
2960/2960 - 44s - loss: -6.1672e-01 - binary_accuracy: 0.9829 - val_loss: -7.1391e-01 - val_binary_accuracy: 0.9937
Epoch 2/20
2960/2960 - 44s - loss: -7.3468e-01 - binary_accuracy: 0.9906 - val_loss: -7.7221e-01 - val_binary_accuracy: 0.9928
Epoch 3/20
2960/2960 - 43s - loss: -7.5273e-01 - binary_accuracy: 0.9917 - val_loss: -7.7431e-01 - val_binary_accuracy: 0.9930
Epoch 4/20
2960/2960 - 44s - loss: -7.6389e-01 - binary_accuracy: 0.9923 - val_loss: -7.9482e-01 - val_binary_accuracy: 0.9927
Epoch 5/20
2960/2960 - 42s - loss: -7.7223e-01 - binary_accuracy: 0.9928 - val_loss: -7.9429e-01 - val_binary_accuracy: 0.9936
Epoch 6/20
2960/2960 - 43s - loss: -7.7735e-01 - binary_accuracy: 0.9930 - val_loss: -8.0574e-01 - val_binary_accuracy: 0.9932
Epoch 7/20
2960/2960 - 44s - loss: -7.8169e-01 - binary_accuracy: 0.9933 - val_loss: -7.9159e-01 - val_binary_accuracy: 0.9940
Epoch 

In [9]:
folder_name = list(model.keys())

util.save_model(model, model_path, ['model']*len(model), folder_name)
util.save_pkl(history, model_path, ['history']*len(model), folder_name)
util.save_pkl(frames_test, model_path, ['frames_test']*len(model), folder_name)


Saving D:/Santi/Models/multich_ref_f8_c9_b8/model.h5

Saving D:/Santi/Models/multich_ref_f8_c9_b16/model.h5

Saving D:/Santi/Models/multich_ref_f8_c9_b32/model.h5

Saving D:/Santi/Models/multich_ref_f8_c9_b256/model.h5
Done.

Saving D:/Santi/Models/multich_ref_f8_c9_b8/history

Saving D:/Santi/Models/multich_ref_f8_c9_b16/history

Saving D:/Santi/Models/multich_ref_f8_c9_b32/history

Saving D:/Santi/Models/multich_ref_f8_c9_b256/history
Done.

Saving D:/Santi/Models/multich_ref_f8_c9_b8/frames_test

Saving D:/Santi/Models/multich_ref_f8_c9_b16/frames_test

Saving D:/Santi/Models/multich_ref_f8_c9_b32/frames_test

Saving D:/Santi/Models/multich_ref_f8_c9_b256/frames_test
Done.


### Mito & WatProc, different gaussian dilations

#### Only mito channel as input

In [23]:
#Inputs
input_data = util.load_h5(data_path, 'Mito')
print('Inputs'+':', input_data.shape)


Loading D:/Santi/Data/Mito.h5
Converting to array
Inputs: (37000, 128, 128)


In [26]:
with gpu:
  nb_filters = 8
  firstConvSize = 9
  batch_size = 16
  
  optimal_sigma = util.load_pkl(data_path, 'max_optimal_sigma')
  threshold = util.load_pkl(data_path, 'max_intensity_threshold')
  model, history, frames_test = {}, {}, {}
  
  for s, t in zip(optimal_sigma, threshold):
    model_name = 'wp_f%i_c%i_b%i_s%.1f_t%.i'%(nb_filters, firstConvSize, batch_size, s, t)
    print('Model:', model_name)
    #Outputs
    output_data = util.load_h5(data_path, 'WatProc_s%.1f_t%.i'%(s, t))
    print('Outputs:', output_data.shape)
    model[model_name] = training.create_model(nb_filters, firstConvSize)
    
    history[model_name], frames_test[model_name] = training.train_model(model[model_name], input_data, output_data, batch_size=batch_size) 


Loading D:/Santi/Data/max_optimal_sigma

Loading D:/Santi/Data/max_intensity_threshold
Model: wp_f8_c9_b16_s1.8_t1.0

Loading D:/Santi/Data/WatProc_s1.8_t1.h5
Converting to array
Outputs: (37000, 128, 128)
* Start Encoder Section *
* Start Center Section *
* Start Decoder Section *
Epoch 1/20
1480/1480 - 28s - loss: 0.1905 - binary_accuracy: 0.9816 - val_loss: 0.0311 - val_binary_accuracy: 0.9967
Epoch 2/20
1480/1480 - 28s - loss: 0.0152 - binary_accuracy: 0.9967 - val_loss: 0.0072 - val_binary_accuracy: 0.9967
Epoch 3/20
1480/1480 - 28s - loss: 0.0050 - binary_accuracy: 0.9967 - val_loss: 0.0038 - val_binary_accuracy: 0.9967
Epoch 4/20
1480/1480 - 28s - loss: 0.0029 - binary_accuracy: 0.9967 - val_loss: 0.0026 - val_binary_accuracy: 0.9967
Epoch 5/20
1480/1480 - 28s - loss: 0.0022 - binary_accuracy: 0.9967 - val_loss: 0.0021 - val_binary_accuracy: 0.9967
Epoch 6/20
1480/1480 - 28s - loss: 0.0019 - binary_accuracy: 0.9967 - val_loss: 0.0019 - val_binary_accuracy: 0.9967
Epoch 7/20
148

In [27]:
folder_name = list(model.keys())

util.save_model(model, model_path, ['model']*len(model), folder_name)
util.save_pkl(history, model_path, ['history']*len(model), folder_name)
util.save_pkl(frames_test, model_path, ['frames_test']*len(model), folder_name)


Saving D:/Santi/Models/wp_f8_c9_b16_s1.8_t1.0/model.h5

Saving D:/Santi/Models/wp_f8_c9_b16_s1.9_t6.0/model.h5

Saving D:/Santi/Models/wp_f8_c9_b16_s2.0_t21.0/model.h5

Saving D:/Santi/Models/wp_f8_c9_b16_s2.1_t33.0/model.h5

Saving D:/Santi/Models/wp_f8_c9_b16_s2.2_t48.0/model.h5

Saving D:/Santi/Models/wp_f8_c9_b16_s2.2_t68.0/model.h5

Saving D:/Santi/Models/wp_f8_c9_b16_s2.3_t98.0/model.h5
Done.

Saving D:/Santi/Models/wp_f8_c9_b16_s1.8_t1.0/history

Saving D:/Santi/Models/wp_f8_c9_b16_s1.9_t6.0/history

Saving D:/Santi/Models/wp_f8_c9_b16_s2.0_t21.0/history

Saving D:/Santi/Models/wp_f8_c9_b16_s2.1_t33.0/history

Saving D:/Santi/Models/wp_f8_c9_b16_s2.2_t48.0/history

Saving D:/Santi/Models/wp_f8_c9_b16_s2.2_t68.0/history

Saving D:/Santi/Models/wp_f8_c9_b16_s2.3_t98.0/history
Done.

Saving D:/Santi/Models/wp_f8_c9_b16_s1.8_t1.0/frames_test

Saving D:/Santi/Models/wp_f8_c9_b16_s1.9_t6.0/frames_test

Saving D:/Santi/Models/wp_f8_c9_b16_s2.0_t21.0/frames_test

Saving D:/Santi/Models

In [28]:
del input_data, output_data

#### Mito + Drp1 channels as inputs

In [10]:
#Inputs
input_data = util.load_h5(data_path, 'Mito')
input_data = np.stack((input_data, util.load_h5(data_path, 'Drp1')), axis=-1)
print('Inputs'+':', input_data.shape)


Loading D:/Santi/Data/Mito.h5
Converting to array

Loading D:/Santi/Data/Drp1.h5
Converting to array
Inputs: (37000, 128, 128, 2)


In [11]:
with gpu:
  nb_filters = 8
  firstConvSize = 9
  nb_input_channels = 2
  batch_size = 16
  
  optimal_sigma = util.load_pkl(data_path, 'max_optimal_sigma')
  threshold = util.load_pkl(data_path, 'max_intensity_threshold')
  model, history, frames_test = {}, {}, {}
  
  for s, t in zip(optimal_sigma, threshold):
    model_name = 'multich_wp_f%i_c%i_b%i_s%.1f_t%.i'%(nb_filters, firstConvSize, batch_size, s, t)
    print('Model:', model_name)
    #Outputs
    output_data = util.load_h5(data_path, 'WatProc_s%.1f_t%.i'%(s, t))
    print('Outputs:', output_data.shape)
    model[model_name] = training.create_model(nb_filters, firstConvSize, nb_input_channels)
    
    history[model_name], frames_test[model_name] = training.train_model(model[model_name], input_data, output_data, batch_size=batch_size) 


Loading D:/Santi/Data/max_optimal_sigma

Loading D:/Santi/Data/max_intensity_threshold
Model: multich_wp_f8_c9_b16_s1.8_t1

Loading D:/Santi/Data/WatProc_s1.8_t1.h5
Converting to array
Outputs: (37000, 128, 128)
* Start Encoder Section *
* Start Center Section *
* Start Decoder Section *
Epoch 1/20
1480/1480 - 29s - loss: 0.1801 - binary_accuracy: 0.9856 - val_loss: 0.0238 - val_binary_accuracy: 0.9967
Epoch 2/20
1480/1480 - 29s - loss: 0.0119 - binary_accuracy: 0.9967 - val_loss: 0.0054 - val_binary_accuracy: 0.9967
Epoch 3/20
1480/1480 - 29s - loss: 0.0037 - binary_accuracy: 0.9967 - val_loss: 0.0026 - val_binary_accuracy: 0.9967
Epoch 4/20
1480/1480 - 29s - loss: 0.0021 - binary_accuracy: 0.9967 - val_loss: 0.0018 - val_binary_accuracy: 0.9967
Epoch 5/20
1480/1480 - 29s - loss: 0.0016 - binary_accuracy: 0.9967 - val_loss: 0.0015 - val_binary_accuracy: 0.9967
Epoch 6/20
1480/1480 - 29s - loss: 0.0014 - binary_accuracy: 0.9967 - val_loss: 0.0014 - val_binary_accuracy: 0.9967
Epoch 7/

In [12]:
folder_name = list(model.keys())

util.save_model(model, model_path, ['model']*len(model), folder_name)
util.save_pkl(history, model_path, ['history']*len(model), folder_name)
util.save_pkl(frames_test, model_path, ['frames_test']*len(model), folder_name)


Saving D:/Santi/Models/multich_wp_f8_c9_b16_s1.8_t1/model.h5

Saving D:/Santi/Models/multich_wp_f8_c9_b16_s1.9_t6/model.h5

Saving D:/Santi/Models/multich_wp_f8_c9_b16_s2.0_t21/model.h5

Saving D:/Santi/Models/multich_wp_f8_c9_b16_s2.1_t33/model.h5

Saving D:/Santi/Models/multich_wp_f8_c9_b16_s2.2_t48/model.h5

Saving D:/Santi/Models/multich_wp_f8_c9_b16_s2.2_t68/model.h5

Saving D:/Santi/Models/multich_wp_f8_c9_b16_s2.3_t98/model.h5
Done.

Saving D:/Santi/Models/multich_wp_f8_c9_b16_s1.8_t1/history

Saving D:/Santi/Models/multich_wp_f8_c9_b16_s1.9_t6/history

Saving D:/Santi/Models/multich_wp_f8_c9_b16_s2.0_t21/history

Saving D:/Santi/Models/multich_wp_f8_c9_b16_s2.1_t33/history

Saving D:/Santi/Models/multich_wp_f8_c9_b16_s2.2_t48/history

Saving D:/Santi/Models/multich_wp_f8_c9_b16_s2.2_t68/history

Saving D:/Santi/Models/multich_wp_f8_c9_b16_s2.3_t98/history
Done.

Saving D:/Santi/Models/multich_wp_f8_c9_b16_s1.8_t1/frames_test

Saving D:/Santi/Models/multich_wp_f8_c9_b16_s1.9_t6

In [13]:
del input_data, output_data

## Spatiotemporal gaussian filter

### Only mito channel as input

In [4]:
#Inputs
input_data = util.load_h5(data_path, 'Mito')
print('Inputs'+':', input_data.shape)


Loading D:/Santi/Data/Mito.h5
Converting to array
Inputs: (37000, 128, 128)


In [6]:
with gpu:
  nb_filters = 8
  firstConvSize = 9
  batch_size = 16
  
  dilation_sigma = util.load_pkl(data_path, 'max_optimal_sigma')
  dilation_threshold = util.load_pkl(data_path, 'max_intensity_threshold')
  event_score_gaussian_threshold = util.load_pkl(data_path, 'event_score_gaussian_thresholds')
  time_window_size = 5
  
  model, history, frames_test = {}, {}, {}
  
  for sigma, threshold, gaussian_threshold in zip(dilation_sigma, dilation_threshold, event_score_gaussian_threshold):
    metadata = 's%.1f_t%i_w%i_gt%i'%(sigma, threshold, time_window_size, gaussian_threshold)
    model_name = 'spatemp_wp_f%i_c%i_b%i_'%(nb_filters, firstConvSize, batch_size) + metadata
    print('Model:', model_name)

    #Output
    output_data = util.load_h5(data_path, 'spatemp_WatProc_'+metadata)
    print('Output:', output_data.shape)
    model[model_name] = training.create_model(nb_filters, firstConvSize)

    history[model_name], frames_test[model_name] = training.train_model(model[model_name], input_data, output_data, batch_size=batch_size) 
    del output_data


Loading D:/Santi/Data/max_optimal_sigma

Loading D:/Santi/Data/max_intensity_threshold

Loading D:/Santi/Data/event_score_gaussian_thresholds
Model: spatemp_wp_f8_c9_b16_s1.8_t1_w5_gt38

Loading D:/Santi/Data/spatemp_WatProc_s1.8_t1_w5_gt38.h5
Converting to array
Output: (37000, 128, 128)
* Start Encoder Section *
* Start Center Section *
* Start Decoder Section *
Epoch 1/20
1480/1480 - 28s - loss: 0.1017 - binary_accuracy: 0.9964 - val_loss: 0.0203 - val_binary_accuracy: 0.9983
Epoch 2/20
1480/1480 - 28s - loss: 0.0105 - binary_accuracy: 0.9982 - val_loss: 0.0056 - val_binary_accuracy: 0.9983
Epoch 3/20
1480/1480 - 28s - loss: 0.0040 - binary_accuracy: 0.9982 - val_loss: 0.0032 - val_binary_accuracy: 0.9983
Epoch 4/20
1480/1480 - 28s - loss: 0.0024 - binary_accuracy: 0.9982 - val_loss: 0.0019 - val_binary_accuracy: 0.9983
Epoch 5/20
1480/1480 - 28s - loss: 0.0017 - binary_accuracy: 0.9982 - val_loss: 0.0015 - val_binary_accuracy: 0.9983
Epoch 6/20
1480/1480 - 28s - loss: 0.0014 - bin

In [7]:
folder_name = list(model.keys())

util.save_model(model, model_path, ['model']*len(model), folder_name)
util.save_pkl(history, model_path, ['history']*len(model), folder_name)
util.save_pkl(frames_test, model_path, ['frames_test']*len(model), folder_name)


Saving D:/Santi/Models/spatemp_wp_f8_c9_b16_s1.8_t1_w5_gt38/model.h5

Saving D:/Santi/Models/spatemp_wp_f8_c9_b16_s1.9_t6_w5_gt49/model.h5

Saving D:/Santi/Models/spatemp_wp_f8_c9_b16_s2.0_t21_w5_gt55/model.h5

Saving D:/Santi/Models/spatemp_wp_f8_c9_b16_s2.1_t33_w5_gt72/model.h5

Saving D:/Santi/Models/spatemp_wp_f8_c9_b16_s2.2_t48_w5_gt91/model.h5

Saving D:/Santi/Models/spatemp_wp_f8_c9_b16_s2.2_t68_w5_gt109/model.h5

Saving D:/Santi/Models/spatemp_wp_f8_c9_b16_s2.3_t98_w5_gt127/model.h5
Done.

Saving D:/Santi/Models/spatemp_wp_f8_c9_b16_s1.8_t1_w5_gt38/history

Saving D:/Santi/Models/spatemp_wp_f8_c9_b16_s1.9_t6_w5_gt49/history

Saving D:/Santi/Models/spatemp_wp_f8_c9_b16_s2.0_t21_w5_gt55/history

Saving D:/Santi/Models/spatemp_wp_f8_c9_b16_s2.1_t33_w5_gt72/history

Saving D:/Santi/Models/spatemp_wp_f8_c9_b16_s2.2_t48_w5_gt91/history

Saving D:/Santi/Models/spatemp_wp_f8_c9_b16_s2.2_t68_w5_gt109/history

Saving D:/Santi/Models/spatemp_wp_f8_c9_b16_s2.3_t98_w5_gt127/history
Done.



### Mito + Drp1 channels as inputs

In [24]:
#Inputs
input_data = util.load_h5(data_path, 'Mito')
input_data = np.stack((input_data, util.load_h5(data_path, 'Drp1')), axis=-1)
print('Inputs'+':', input_data.shape)


Loading D:/Santi/Data/Mito.h5
Converting to array

Loading D:/Santi/Data/Drp1.h5
Converting to array
Inputs: (37000, 128, 128, 2)


In [28]:
with gpu:
  nb_filters = 8
  firstConvSize = 9
  nb_input_channels = 2
  batch_size = 16
  
  dilation_sigma = util.load_pkl(data_path, 'max_optimal_sigma')
  dilation_threshold = util.load_pkl(data_path, 'max_intensity_threshold')
  event_score_gaussian_threshold = util.load_pkl(data_path, 'event_score_gaussian_thresholds')
  time_window_size = 5
  
  model, history, frames_test = {}, {}, {}
  
  for sigma, threshold, gaussian_threshold in zip(dilation_sigma, dilation_threshold, event_score_gaussian_threshold):
    metadata = 's%.1f_t%i_w%i_gt%i'%(sigma, threshold, time_window_size, gaussian_threshold)
    model_name = 'multich_spatemp_wp_f%i_c%i_b%i_'%(nb_filters, firstConvSize, batch_size) + metadata
    print('Model:', model_name)

    #Output
    output_data = util.load_h5(data_path, 'spatemp_WatProc_'+metadata)
    print('Output:', output_data.shape)
    model[model_name] = training.create_model(nb_filters, firstConvSize, nb_input_channels)

    history[model_name], frames_test[model_name] = training.train_model(model[model_name], input_data, output_data, batch_size=batch_size) 
    del output_data


Loading D:/Santi/Data/max_optimal_sigma

Loading D:/Santi/Data/max_intensity_threshold

Loading D:/Santi/Data/event_score_gaussian_thresholds
Model: multich_spatemp_WatProc_spatemp_wp_f8_c9_b16_s1.8_t1_w5_gt38

Loading D:/Santi/Data/spatemp_WatProc_s1.8_t1_w5_gt38.h5
Converting to array
Output: (37000, 128, 128)
* Start Encoder Section *
* Start Center Section *
* Start Decoder Section *
Epoch 1/20
1480/1480 - 29s - loss: 0.0758 - binary_accuracy: 0.9977 - val_loss: 0.0124 - val_binary_accuracy: 0.9982
Epoch 2/20
1480/1480 - 28s - loss: 0.0064 - binary_accuracy: 0.9982 - val_loss: 0.0032 - val_binary_accuracy: 0.9982
Epoch 3/20
1480/1480 - 29s - loss: 0.0023 - binary_accuracy: 0.9982 - val_loss: 0.0017 - val_binary_accuracy: 0.9982
Epoch 4/20
1480/1480 - 28s - loss: 0.0014 - binary_accuracy: 0.9982 - val_loss: 0.0012 - val_binary_accuracy: 0.9982
Epoch 5/20
1480/1480 - 29s - loss: 0.0011 - binary_accuracy: 0.9982 - val_loss: 0.0010 - val_binary_accuracy: 0.9982
Epoch 6/20
1480/1480 - 

In [29]:
folder_name = list(model.keys())

util.save_model(model, model_path, ['model']*len(model), folder_name)
util.save_pkl(history, model_path, ['history']*len(model), folder_name)
util.save_pkl(frames_test, model_path, ['frames_test']*len(model), folder_name)


Saving D:/Santi/Models/multich_spatemp_WatProc_spatemp_wp_f8_c9_b16_s1.8_t1_w5_gt38/model.h5

Saving D:/Santi/Models/multich_spatemp_WatProc_spatemp_wp_f8_c9_b16_s1.9_t6_w5_gt49/model.h5

Saving D:/Santi/Models/multich_spatemp_WatProc_spatemp_wp_f8_c9_b16_s2.0_t21_w5_gt55/model.h5

Saving D:/Santi/Models/multich_spatemp_WatProc_spatemp_wp_f8_c9_b16_s2.1_t33_w5_gt72/model.h5

Saving D:/Santi/Models/multich_spatemp_WatProc_spatemp_wp_f8_c9_b16_s2.2_t48_w5_gt91/model.h5

Saving D:/Santi/Models/multich_spatemp_WatProc_spatemp_wp_f8_c9_b16_s2.2_t68_w5_gt109/model.h5

Saving D:/Santi/Models/multich_spatemp_WatProc_spatemp_wp_f8_c9_b16_s2.3_t98_w5_gt127/model.h5
Done.

Saving D:/Santi/Models/multich_spatemp_WatProc_spatemp_wp_f8_c9_b16_s1.8_t1_w5_gt38/history

Saving D:/Santi/Models/multich_spatemp_WatProc_spatemp_wp_f8_c9_b16_s1.9_t6_w5_gt49/history

Saving D:/Santi/Models/multich_spatemp_WatProc_spatemp_wp_f8_c9_b16_s2.0_t21_w5_gt55/history

Saving D:/Santi/Models/multich_spatemp_WatProc_s